In [1]:
def _is_notebook():
    try:
        from IPython import get_ipython

        return "IPKernelApp" in get_ipython().config
    except Exception:
        return False

In [2]:
from dotenv import load_dotenv

assert load_dotenv("../../.env")

In [3]:
import ragna

ragna.__version__

'0.1.dev16+ga4d3603.d20230914143851'

In [4]:
from ragna import demo_config, Config

demo_config

Config(state_database_url='sqlite://', queue_database_url='redis://127.0.0.1:6379', ragna_api_url='http://127.0.0.1:31476', ragna_ui_url='http://127.0.0.1:31477', local_cache_root=PosixPath('/home/philip/.cache/ragna'), document_class=<class 'ragna.core.LocalDocument'>, upload_token_secret='9e6020416528a671c3057800c6ce4452b6f77e9b29f62647fa9a8fb855065f54', upload_token_ttl=30, registered_source_storage_classes={'Ragna/DemoSourceStorage': <class 'ragna.source_storage._demo.RagnaDemoSourceStorage'>}, registered_assistant_classes={'Ragna/DemoAssistant': <class 'ragna.assistant._demo.RagnaDemoAssistant'>})

In [5]:
document_path = "demo_document.txt"

with open(document_path, "w") as file:
    file.write("Ragna is an open-source RAG orchestration app\n")

In [6]:
from ragna.core import Rag
from ragna.assistant import (
    RagnaDemoAssistant,
    OpenaiGpt35Turbo16kAssistant,
    OpenaiGpt4Assistant,
)
from ragna.source_storage import ChromaSourceStorage, RagnaDemoSourceStorage

rag = Rag(demo_config)

async with await rag.new_chat(
    documents=[document_path],
    source_storage=RagnaDemoSourceStorage,
    assistant=OpenaiGpt35Turbo16kAssistant,
) as chat:
    prompt = "What is Ragna?"
    answer = await chat.answer(prompt)

print(answer)

{"event": "Started ragna worker", "timestamp": "2023-09-20T19:14:01.928952Z", "level": "info"}
Ragna is an open-source RAG orchestration app. It is designed to help users create and manage RAG (Red, Amber, Green) status reports. Ragna allows users to easily track and communicate the status of various projects or tasks using a simple color-coded system. It provides a user-friendly interface for creating and updating RAG reports, as well as generating visualizations and reports based on the data entered. Ragna is built on open-source technologies and is available for free to use and customize.


In [7]:
import itertools
import asyncio
from pprint import pprint

source_storages = [ChromaSourceStorage]
assistants = [OpenaiGpt35Turbo16kAssistant, OpenaiGpt4Assistant]


async def answer_prompt(source_storage, assistant):
    chat = await rag.new_chat(
        documents=[document_path],
        source_storage=source_storage,
        assistant=assistant,
    )
    await chat.start()
    return await chat.answer(prompt)


experiments = {
    (source_storage.display_name(), assistant.display_name()): answer_prompt(
        source_storage, assistant
    )
    for source_storage, assistant in itertools.product(source_storages, assistants)
}

pprint(experiments)

{('Chroma', 'OpenAI/gpt-3.5-turbo-16k'): <coroutine object answer_prompt at 0x7f151a0e7240>,
 ('Chroma', 'OpenAI/gpt-4'): <coroutine object answer_prompt at 0x7f151a0e7440>}


In [8]:
results = dict(zip(experiments.keys(), await asyncio.gather(*experiments.values())))
pprint(results)

{('Chroma', 'OpenAI/gpt-3.5-turbo-16k'): Ragna is an open-source RAG (Response Analysis Graph) orchestration app. It is designed to help users create conversational AI applications by providing a framework for managing and orchestrating the flow of conversations. Ragna allows developers to define conversation flows, handle user inputs, and generate dynamic responses based on predefined rules and logic. It is built on top of the Rasa framework and provides additional features and functionalities to simplify the development process.,
 ('Chroma', 'OpenAI/gpt-4'): Ragna is an open-source RAG orchestration app.}
